## Data Analysis using BulkLMM - BXD Spleen Study

In [23]:
using CSV, DelimitedFiles, DataFrames, Missings, XLSX
using LinearAlgebra, Statistics, Optim
using Random, Distributions, LoopVectorization
using BenchmarkTools

In [24]:
using Plots

In [25]:
versioninfo()

Julia Version 1.9.2
Commit e4ee485e909 (2023-07-05 09:39 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 48 × Intel(R) Xeon(R) Silver 4214 CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, cascadelake)
  Threads: 17 on 48 virtual cores
Environment:
  JULIA_NUM_THREADS = 16


In [26]:
local_path = "../../BulkLMM.jl/src";

In [111]:
include(joinpath(local_path, "kinship.jl"));
include(joinpath(local_path, "util.jl"));
include(joinpath(local_path, "wls.jl"));
include(joinpath(local_path, "lmm.jl"));
include(joinpath(local_path, "gridbrent.jl"));
include(joinpath(local_path, "transform_helpers.jl"));
include(joinpath(local_path, "scan.jl"));
include(joinpath(local_path, "bulkscan_helpers.jl"));
include(joinpath(local_path, "bulkscan.jl"));
include(joinpath(local_path, "readData.jl"));
include(joinpath(local_path, "analysis_helpers/single_trait_analysis.jl"));

In [28]:
include(joinpath(local_path, "../test/testHelpers.jl"));

### Load data:

In [29]:
bulklmmdir = local_path;
pheno_file = joinpath(bulklmmdir,"..","data/bxdData/spleen-pheno-nomissing.csv");
pheno = readdlm(pheno_file, ',', header = false);
pheno_processed = pheno[2:end, 2:(end-1)].*1.0; # exclude the header, the first (transcript ID)and the last columns (sex)

In [30]:
geno_file = joinpath(bulklmmdir,"..","data/bxdData/spleen-bxd-genoprob.csv");
geno = readdlm(geno_file, ',', header = false);
geno_processed = geno[2:end, 1:2:end] .* 1.0;

In [31]:
size(pheno_processed) # (number of strains, number of traits)

(79, 35554)

In [32]:
size(geno_processed) # (number of strains, number of traits)

(79, 7321)

In [33]:
@time kinship = calcKinship(geno_processed); # calculate K

  0.004578 seconds (8 allocations: 4.508 MiB)


In [ ]:
using Helium

In [ ]:
kinship = Helium.readhe(joinpath(local_path, "../test/ref_data_for_tests/kinship_ref.he"));

In [ ]:
kinship

In [19]:
BLAS.get_num_threads()

24

In [ ]:
BLAS.set_num_threads(4)

In [ ]:
(D, U) = eigen(kinship);

In [ ]:
U

In [ ]:
D

In [ ]:
U

### Single trait scans:

In [138]:
traitID = 1112;
pheno_y = reshape(pheno_processed[:, traitID], :, 1);

In [139]:
@time single_results = scan(pheno_y, geno_processed, kinship);

  0.060065 seconds (80.96 k allocations: 47.299 MiB)


In [140]:
single_results.h2_null

0.8500907448548001

In [141]:
@time single_results_perms = scan(pheno_y, geno_processed, kinship; permutation_test = true, nperms = 1000);

  0.068068 seconds (90.09 k allocations: 146.533 MiB)


In [142]:
single_results_perms.lod

7321-element Vector{Float64}:
 0.3776050818288771
 0.37760508182887126
 0.37760508182887126
 0.37760508182887126
 0.3776050818288771
 0.3776050818288732
 0.37760508182887514
 0.37760508182887514
 0.37760508183021657
 0.37760508183021657
 0.37760539012006483
 0.3777720004833044
 0.5735036416999362
 ⋮
 0.00018149541861298162
 0.00018149133384645852
 0.00018149133384455393
 0.012408223483856767
 0.07976320868738593
 0.079763208687627
 0.38873354088127104
 0.39210072030207904
 0.39211667483269136
 0.3818705225343384
 0.6715215850741068
 0.6715215850742929

In [143]:
single_results_perms.L_perms

7321×1000 Matrix{Float64}:
 0.0145766  0.132362  0.881491   …  0.59105     0.438842   0.206609
 0.0145766  0.132362  0.881491      0.59105     0.438842   0.206609
 0.0145766  0.132362  0.881491      0.59105     0.438842   0.206609
 0.0145766  0.132362  0.881491      0.59105     0.438842   0.206609
 0.0145766  0.132362  0.881491      0.59105     0.438842   0.206609
 0.0145766  0.132362  0.881491   …  0.59105     0.438842   0.206609
 0.0145766  0.132362  0.881491      0.59105     0.438842   0.206609
 0.0145766  0.132362  0.881491      0.59105     0.438842   0.206609
 0.0145766  0.132362  0.881491      0.59105     0.438842   0.206609
 0.0145766  0.132362  0.881491      0.59105     0.438842   0.206609
 0.0145766  0.132362  0.88149    …  0.591051    0.438841   0.206609
 0.0145899  0.132374  0.881268      0.591437    0.438773   0.20665
 0.0367593  0.124242  0.447157      1.14994     0.269512   0.227333
 ⋮                               ⋱                         
 0.0427627  1.12733   0.060041

In [144]:
thrs = get_thresholds(single_results_perms.L_perms, [0.90, 0.95]).thrs
round.(thrs; digits = 4)

2-element Vector{Float64}:
 3.3854
 3.6745

### Multiple trait scans:

In [20]:
Threads.nthreads()

16

In [21]:
BLAS.get_num_threads()

24

In [17]:
BLAS.set_num_threads(4)

In [18]:
h2_grid = collect(0.0:0.01:0.99);

In [19]:
h2_grid2 = collect(0.0:0.05:0.95);

In [20]:
pheno_st = colStandardize(pheno_processed);

In [91]:
@benchmark bulkscan(pheno_processed, geno_processed, kinship)

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.910 s …   2.038 s  ┊ GC (min … max): 0.00% … 0.66%
 Time  (median):     1.919 s              ┊ GC (median):    0.68%
 Time  (mean ± σ):   1.956 s ± 71.492 ms  ┊ GC (mean ± σ):  0.45% ± 0.39%

  █   █                                                   █  
  █▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.91 s         Histogram: frequency by time        2.04 s <

 Memory estimate: 5.05 GiB, allocs estimate: 107941.

In [115]:
@time all_results_grid = bulkscan(pheno_processed, geno_processed, kinship; output_pvals = true);

 11.429474 seconds (1.29 M allocations: 7.066 GiB, 0.55% gc time, 26.63% compilation time: <1% of which was recompilation)


In [116]:
all_results_grid

(L = [0.00012008682100828602 0.04714302466927942 … 0.003349166892450877 0.34869129344849176; 0.00012008682100828602 0.04714302466927942 … 0.003349166892450877 0.348691293448486; … ; 0.18170671563925866 0.2541086612310956 … 0.05471098554541802 0.06336333569432846; 0.18170671563924712 0.254108661231171 … 0.05471098554541038 0.06336333569427495], h2_null_list = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.7, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.7, 0.0, 0.0], Pvals_mat = [0.9812383707202973 0.6412572440184201 … 0.9011637960803418 0.20508563231731666; 0.9812383707202973 0.6412572440184201 … 0.9011637960803418 0.2050856323173204; … ; 0.3603163488549547 0.2793573495051721 … 0.615702975094421 0.5890700240772564; 0.36031634885497 0.2793573495051007 … 0.6157029750944456 0.5890700240774136], Chisq_df = 1)

In [112]:
-log(10, lod2p(3.0, 2))

3.0

In [113]:
function lod2logP(LODs::Union{Array{Float64,1},Array{Any,1}},v::Int64)
return -log.(10,(ccdf.(Chisq(v),2*log(10)*LODs)))
end

lod2logP (generic function with 1 method)

In [114]:
lod2logP([3.0], 2)

1-element Vector{Float64}:
 3.0

In [96]:
size(all_results_grid.L)

(7321, 35554)

In [95]:
size(all_results_grid.Pvals_mat)

(7321, 35554)

In [24]:
@time all_results_grid_st = bulkscan_null_grid(pheno_st, geno_processed, kinship, h2_grid;
                                               prior_variance = 1.0, prior_sample_size = 0.1);

 10.736517 seconds (49.06 M allocations: 17.617 GiB, 11.09% gc time)


In [25]:
@time all_results_exact = bulkscan_null(pheno_processed, geno_processed, kinship;
                                        optim_interval = 1);

 86.247687 seconds (2.86 G allocations: 706.927 GiB, 36.87% gc time, 0.07% compilation time)


In [26]:
@time all_results_exact_st = bulkscan_null(pheno_st, geno_processed, kinship;
                                           prior_variance = 1.0, prior_sample_size = 0.1,
                                           optim_interval = 1);

 76.416246 seconds (2.86 G allocations: 706.924 GiB, 37.34% gc time)


In [27]:
@time all_results_alt_grid = bulkscan_alt_grid(pheno_processed, geno_processed, kinship, h2_grid);

701.758396 seconds (90.10 M allocations: 810.950 GiB, 16.24% gc time, 0.03% compilation time)


In [28]:
@time all_results_alt_grid2 = bulkscan_alt_grid(pheno_processed, geno_processed, kinship, h2_grid2);

145.839825 seconds (18.27 M allocations: 171.527 GiB, 16.44% gc time)


In [32]:
findall(all_results_grid.h2_null_list .> 0.0)

3646-element Vector{Int64}:
    82
    95
   107
   108
   128
   153
   234
   253
   258
   298
   348
   354
   381
     ⋮
 35495
 35501
 35509
 35516
 35526
 35527
 35529
 35534
 35535
 35545
 35548
 35552

In [34]:
all_results_grid.h2_null_list[82]

0.78

In [35]:
all_results_alt_grid.h2_panel[:, 82]

7321-element Vector{Float64}:
 0.57
 0.57
 0.57
 0.57
 0.57
 0.57
 0.57
 0.57
 0.57
 0.57
 0.57
 0.57
 0.56
 ⋮
 0.57
 0.57
 0.57
 0.57
 0.59
 0.59
 0.58
 0.59
 0.59
 0.6
 0.59
 0.59

In [29]:
all_results_alt_grid.L[:, 1]

7321-element Vector{Float64}:
 0.00012008682101093471
 0.00012008682101093471
 0.00012008682101093471
 0.00012008682101093471
 0.00012008682101093471
 0.00012008682101093471
 0.00012008682101093471
 0.00012008682101093471
 0.00012008682099859131
 0.00012008682099859131
 0.00012008387637761301
 0.00011849771856038352
 0.009521311901900588
 ⋮
 0.26017848855422443
 0.26017851503361367
 0.26017851503361983
 0.2534463880193611
 0.3782893583076069
 0.37828935830770566
 0.21935139991554947
 0.20144120787677675
 0.20153298758761384
 0.17782339280604786
 0.18170671563925148
 0.18170671563923912

In [36]:
include("../../BigRiver_util_code/src/kinship_utils.jl");
include("../../BigRiver_util_code/src/run_gemma_utils.jl");

In [37]:
pwd()

"/home/zyu20/git/BulkLMM_Analyses/BXDSpleen"

In [41]:
gmap = CSV.read("../../BulkLMM.jl/data/bxdData/gmap.csv", DataFrame);

In [44]:
marker_names = gmap.Locus |> x -> String.(x) |> x -> Array{String, 1}(x);

In [45]:
pheno_filename = "data/GEMMA_data/bxd_spleen_pheno.txt";
geno_filename = "data/GEMMA_data/bxd_spleen_geno.txt";
kinship_filename = "data/GEMMA_data/bxd_spleen_kinship.txt";
output_filename = "results_univariate_LMM";

In [48]:
gemma = "/home/zyu20/Softwares/gemma-0.98.5-linux-static-AMD64";

In [49]:
@time gemma_one_trait_results = run_gemma(reshape(pheno_processed[:, 82], :, 1), geno_processed, kinship,
                                        ["A", "B"], marker_names,
                                        pheno_filename, geno_filename, kinship_filename, 
                                        output_filename, 
                                        gemma);

GEMMA 0.98.5 (2021-08-25) by Xiang Zhou, Pjotr Prins and team (C) 2012-2021
Reading Files ... 
## number of total individuals = 79
## number of analyzed individuals = 79
## number of covariates = 1
## number of phenotypes = 1
## number of total SNPs/var        =     7321
## number of analyzed SNPs         =     7321
Start Eigen-Decomposition...
pve estimate =0.677983
se(pve) =0.202993
================================================== 100%


**** INFO: Done.


  2.741495 seconds (3.83 M allocations: 329.091 MiB, 38.53% gc time, 10.02% compilation time)


In [53]:
all_results_exact.h2_null_list[82]

0.7817412174810534

In [55]:
@time test_single_trait = scan(reshape(pheno_processed[:, 82], :, 1), geno_processed, kinship);

  0.083102 seconds (81.13 k allocations: 47.366 MiB, 24.75% gc time)


In [56]:
test_single_trait.h2_null

0.7817412174810534

In [58]:
hcat(gemma_one_trait_results, test_single_trait.lod, all_results_exact.L[:, 82], all_results_grid.L[:, 82])

7321×4 Matrix{Float64}:
 0.356661    0.376053     0.376053     0.376463
 0.356661    0.376053     0.376053     0.376463
 0.356661    0.376053     0.376053     0.376463
 0.356661    0.376053     0.376053     0.376463
 0.356661    0.376053     0.376053     0.376463
 0.356661    0.376053     0.376053     0.376463
 0.356661    0.376053     0.376053     0.376463
 0.356661    0.376053     0.376053     0.376463
 0.356661    0.376053     0.376053     0.376463
 0.356661    0.376053     0.376053     0.376463
 0.356661    0.376052     0.376052     0.376463
 0.356494    0.375938     0.375938     0.376349
 0.0715195   0.176568     0.176568     0.177043
 ⋮                                     
 0.00111885  0.0029897    0.0029897    0.0030035
 0.00111885  0.0029897    0.0029897    0.00300349
 0.00111885  0.0029897    0.0029897    0.00300349
 3.41986e-5  0.000736934  0.000736934  0.000756148
 0.248875    0.0381599    0.0381599    0.0377643
 0.248875    0.0381599    0.0381599    0.0377643
 0.199677    0

In [62]:
function meanAbsDiff(x, y)
    
    return mean(abs.(x .- y))
    
end

meanAbsDiff (generic function with 1 method)

In [63]:
meanAbsDiff(gemma_one_trait_results, test_single_trait.lod)

0.10815214516551538

In [65]:
findmax(abs.(gemma_one_trait_results .- test_single_trait.lod))

(0.8248290873637598, CartesianIndex(4918, 1))

In [66]:
gemma_one_trait_results[4918]

1.9718690158524306

In [67]:
test_single_trait.lod[4918]

1.1470399284886708

In [ ]:
plot()

In [60]:
findall(isnan.(gemma_one_trait_results))

CartesianIndex{2}[]

In [61]:
findall(isinf.(gemma_one_trait_results))

CartesianIndex{2}[]